@Abdulrahman Alabrash 

https://github.com/alabrashJr/DCNN-Julia


Read data from saved file 
load("TREC_sib.jld2","datas")-> returns revs, W, W2, word_idx_map, vocab
* revs Dict{String,Any} with 5 entries:y,num_words,tree,text,split
    * y-> label of the questions 1-5 
    * num_words-> length of questions 
    * tree -> 46-element Array{Array,1}, each array contains data_sibling(5)+data_tree(5) =10
    * text -> the question text 
    * split -> type of tuple (training, test , div)



* W wordembedding generated by google2vec, size(10098×300)
* W2 word embedding generated by -0.25 , 0.25 size(10098×300)
* word_idx_map Dict{Any,Any} with 10097 entries , unique word id dictionary 
* vocab DefaultDict{Any,Any,Int64} with 10097 entries, vocab and reptation dictionary

In [1]:
using Pkg;Pkg.update(); for p in ("Embeddings","DataFrames","DataStructures","DataFrames","FileIO","LinearAlgebra","Knet","FileIO"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using DataStructures,DataFrames,FileIO,Embeddings,LinearAlgebra,DataFrames;
using Base.Iterators: flatten
using Statistics: mean
using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, progress, sgd, param, param0, dropout, relu, Data,minibatch;

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [2]:
labels=["abbreviation", "entity", "description", "location" ,"numeric"," "]
revs, W, W2, word_idx_map, vocab=load("Data/TREC_sib.jld2","datas"); 
word_idx_map["ROOT"]=size(W,1);
W=W';

In [3]:
#change W2 with prof's method 
# struct Embed; w; end
# Embed(vocabsize::Int,embedsize::Int) = Embed(param(embedsize,vocabsize))
# (e::Embed)(x) = e.w[:,x]
# W2=Embed(300,length(vocab))

In [4]:
#Transforms sentence into a list of indices. Pad with zeroes.
function get_text_mat(t,word_idx_map;max_l=56,filter_h=5)
    #t the text of question
    x=[] # output matrix
    pad=filter_h -1 # padding number
    for i in collect(1:pad);push!(x,0);end #adding padding 
    words=split(t)
    #extract the unique id of words in the question text and adding it to the matrix 
    for w in words
        if w in keys(word_idx_map);push!(x,word_idx_map[w])
        else; @show w ;end
    end    

    while length(x)<max_l+2*pad    # accomplish 64 +1 size by adding zeros till finish 
            push!(x,0)
    end
    
    return  x
end

get_text_mat (generic function with 1 method)

In [5]:
function getSen(vector)
#labels=["abbreviation","numeric",  "description", "human","location" ,"entity"]
t=Array{Int}(vector)
println(permutedims(t))
for i in t
    if i==1557;print("?\n y =",t[end]+1 );break;end
    if i==0;continue;end
    for (key,value) in word_idx_map
        if value==i; print(key," ");end
    end
end
end

getSen (generic function with 1 method)

In [6]:
#Transforms sentence into a list of indices. Pad with zeroes. 
function get_tree_rep(r,word_idx_map)
# question 
#@show t=r["tree"] #the tree of question
    each_sent=deepcopy(r)# output matrix
    for (j, each_word) in enumerate(each_sent[1:end-1])
        #@show (j, each_word)
            for (l, each_field) in enumerate(each_word)
           # @show (l, each_field)
                if each_field in keys( word_idx_map)
                #@show j,l ;
                    each_sent[j]=Array{Any,1}(each_sent[j])
                     each_sent[j][l] = word_idx_map[each_field]
                elseif each_field == 0
                    continue
                else
                    @show each_field
                end
            end
    end       
    return each_sent;
end

get_tree_rep (generic function with 1 method)

In [7]:
ran=rand(1:length(revs));@show ran
print(revs[ran]["text"]); println("\t",length(split(revs[ran]["text"])))
println(permutedims(get_text_mat(revs[ran]["text"],word_idx_map;max_l=56) ))
for i in split(revs[ran]["text"]) ;println(i,"=",word_idx_map[i]) ;end

ran = 314
What was Einstein 's IQ ?	6
Any[0 0 0 0 6965 1624 6086 8251 2181 1557 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
What=6965
was=1624
Einstein=6086
's=8251
IQ=2181
?=1557


In [8]:
tree_ind=1
println(revs[ran]["tree"][tree_ind])
println(permutedims(get_tree_rep(revs[ran]["tree"],word_idx_map))[tree_ind])
for i in revs[ran]["tree"][tree_ind] ;println(i,"=",word_idx_map[i]) ;end

Any["ROOT", "ROOT", "ROOT", "'s", "Einstein", "'s", "Einstein", "*STOP*", "*STOP*", "IQ"]
Any[10098, 10098, 10098, 8251, 6086, 8251, 6086, 4856, 4856, 2181]
ROOT=10098
ROOT=10098
ROOT=10098
's=8251
Einstein=6086
's=8251
Einstein=6086
*STOP*=4856
*STOP*=4856
IQ=2181


In [9]:
permutedims(vcat(revs[1]["tree"][1:end-1]...))

1×450 Array{Any,2}:
 "ROOT"  "ROOT"  "ROOT"  "did"  "develop"  …  "*ZERO*"  "*ZERO*"  "*ZERO*"

In [10]:
function train_dev_test(revs)
    s1,s2,s3=[],[],[]
    t1,t2,t3=[],[],[]
    for rev in revs
    sent =get_text_mat(rev["text"], word_idx_map)   
    push!(sent,rev["y"])
    sent_tensor = get_tree_rep(rev["tree"], word_idx_map)
        
    if rev["split"]==1
            push!(s1,Array{Int}(sent))
            push!(t1,sent_tensor)
    elseif rev["split"]==2
            push!(s2,Array{Int}(sent))
            push!(t2,sent_tensor)
    elseif rev["split"]==3
            push!(s3,Array{Int}(sent))
            push!(t3,sent_tensor)
    end
end

    train = hcat([f1 for f1 in s1]...)
    test =hcat([f1 for f1 in s2]...)
    dev = hcat([f1 for f1 in s3]...)
    train_tensor = t1
    test_tensor = t2
    dev_tensor = t3
    return (train,test,dev),(train_tensor,test_tensor,dev_tensor)
end
dataset,datasetTensor=train_dev_test(revs);


In [11]:
#try on some vectors 
ran=rand(1:size(dataset[1],2))
getSen(dataset[1][:,ran])

[0 0 0 0 276 1189 9009 2836 4868 9231 9009 6820 4226 8045 7196 129 9003 1557 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3]
Who is the richest person in the world , without owning a business ?
 y =4

dataset[1] --> train 65×5452 } text data
dataset[2]----> test 65×500  }
dataset[3] ----> dev 0x0     }

datasetTensor[1] ---> train_tensor 5452(46(10)))
datasetTensor[2] ---> test_tensor 500(46(10)))
datasetTensor[3] --->dev_tensor 0(0(0))

In [12]:
datasetTensor[2][1]

46-element Array{Any,1}:
 Any[10098, 10098, 10098, 1557, 1679, 1557, 1679, 4856, 4856, 10098] 
 Any[10098, 10098, 1557, 1679, 10098, 1679, 10098, 4856, 4856, 10098]
 Any[10098, 1557, 1679, 10098, 10098, 627, 1679, 8148, 8148, 10098]  
 Any[1557, 1679, 10098, 10098, 10098, 1189, 1679, 627, 8148, 10098]  
 Any[1557, 1679, 10098, 10098, 10098, 9231, 1679, 1557, 4856, 10098] 
 Any[1679, 10098, 10098, 10098, 10098, 6797, 9231, 4856, 4856, 1679] 
 Any[627, 1679, 10098, 10098, 10098, 6965, 627, 8148, 8148, 1679]    
 Any[1189, 1679, 10098, 10098, 10098, 7118, 7118, 7118, 7118, 7118]  
 Any[9231, 1679, 10098, 10098, 10098, 7118, 7118, 7118, 7118, 7118]  
 Any[6797, 9231, 1679, 10098, 10098, 7118, 7118, 7118, 7118, 7118]   
 Any[6965, 627, 1679, 10098, 10098, 7118, 7118, 7118, 7118, 7118]    
 Any[10098, 10098, 10098, 10098, 10098, 7118, 7118, 7118, 7118, 7118]
 Any[10098, 10098, 10098, 10098, 10098, 7118, 7118, 7118, 7118, 7118]
 ⋮                                                               

In [13]:
sent1=vcat([permutedims(vcat(datasetTensor[1][x][1:end-1]...)) for x in 1:size(datasetTensor[1],1)]);#sent1=vcat([permutedims(vcat(datasetTensor[1][1][1:end-1]...)) for x in 1:size(datasetTensor[1],1)]...)
sent2=vcat([permutedims(vcat(datasetTensor[2][x][1:end-1]...)) for x in 1:size(datasetTensor[2],1)]);#sent1=vcat([permutedims(vcat(datasetTensor[1][1][1:end-1]...)) for x in 1:size(datasetTensor[1],1)]...);
#sent1=vcat([permutedims(vcat(datasetTensor[1][1][1:end-1]...)) for x in 1:size(datasetTensor[1],1)]...)'
size(sent1)

(5451,)

In [14]:
y_train=Array{Int8}([dataset[1][:,x][end] for x in 1:size(dataset[1],2)]);#ytrainT=[datasetTensor[1][x,:][end][end][end] for x in 1:size(datasetTensor[1],1)];
y_test=Array{Int8}([dataset[2][:,x][end] for x in 1:size(dataset[2],2)]);#ytestT=[datasetTensor[2][x,:][end][end][end] for x in 1:size(datasetTensor[2],1)];



In [15]:
size(sent1),size(y_train),size(W)

((5451,), (5451,), (300, 10098))

In [16]:
dtrn=minibatch(sent1,y_train,160,shuffle=true);
dtst=minibatch(sent2,y_test,160);
#summary.(Iterators.first(dtrn))
# (x,y) = first(dtrn)
# println.(summary.(x));
summary.(Iterators.first(dtrn))

("160-element Array{Array{Any,2},1}", "160-element Array{Int8,1}")

In [17]:
# Define a convolutional layer:
struct Conv; w; b; f; p;E; end
function (c::Conv)(x) 
#     println("\nConvdeyim \t", typeof(x),"\t", summary(x) )
    #xx=KnetArray{Float32}(reshape(c.E[:, permutedims(hcat(x...))],(300,450,1,160)))
    return c.f.(pool(conv4(c.w, dropout(x,c.p)) .+ c.b))
end
Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0,E=W) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop,E)

Conv

In [18]:
# Redefine dense layer (See mlp.ipynb):
struct Dense; w; b; f; p; end
function (d::Dense)(x) 
#     println("\nDensedeyim ,\t " , typeof(x),"\t", summary(x))
    d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
end
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [19]:
# Let's define a chain of layers
struct Chain
    layers
    Chain(layers...) = new(layers)
end
function (c::Chain)(x) 
    x=KnetArray{Float32}(reshape(W[:, permutedims(hcat(x...))],(300,450,1,160)))
#     println("\nChaindeyim ,\t ", typeof(x),"\t", summary(x))
    (for l in c.layers; x = l(x); end; x)
end
function (c::Chain)(x,y) 
#     println("\nloss Chaindeyim x ,\t ", typeof(x),"\t", summary(x))
#     println("\nloss Chaindeyim y ,\t ", typeof(y),"\t", summary(y))
    nll(c(x),y)
    
end
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)

In [20]:
# hidden_units=[100,2] #which meaning ...... how many conv layers 
# dropout_rate=[0.5] #where in which layer ? conv or dense
n_epochs=20;
# batch_size=170, 
lr_decay = 0.95
function trainresults(file,model; o...)
    if (print("Train from scratch? "); readline()[1]=='y')
        takeevery(n,itr) = (x for (i,x) in enumerate(itr) if i % n == 1)
        r = ((model(dtrn), model(dtst), zeroone(model,dtrn), zeroone(model,dtst))
             for x in takeevery(length(dtrn), progress(sgd(model,repeat(dtrn,n_epochs),lr=lr_decay))))
        r = reshape(collect(Float32,flatten(r)),(4,:))
        Knet.save(file,"results",r)
        Knet.gc() # To save gpu memory
    else
        if isfile(file);r=Knet.load(file,"results");else;println("there is no file such this");return;end
    end
    println(minimum(r,dims=2))
    return r
end

trainresults (generic function with 1 method)

In [21]:
dcnn =   Chain( Conv(3,3,1,1), 
                Conv(4,4,1,1),
                Conv(5,5,1,1),
                Dense(1802,1100,pdrop=0.5), 
                Dense(1100,6,pdrop=0.5))
summary.(l.w for l in dcnn.layers)

#                                                           y                 parameter                 memory
                                                                                                            
#input=300×450                                                                                       300x450
#x1=300-3+1= 298/2 =149 ,y1= 450-3+1= /2 =  224   => y(149,224,5)              3x3x1x5               149x224x5
#x2=149-4+1= 146/2 =73  ,y2= 224-4+1= /2=  110   => y(73,110,10)               4x4x5x10              73x110x10
#x3=73-5+1=  69/2  =34  ,y2= 110-5+1= /2=  53    => y(34,53,10)                5x5x10x10             34x53x10
#fc=34*53*10= 18.020                                                           5x5x10x18020           18.020
#fc=1100x6=6600                                                                18020x1100              6600

# Sum                                                                         ------------          ------------
# #float variable                                                                                    424.820 x4
#Byte                                                                                               =1.699.280
#Batch memory B=160                                                                                 =271.884.800
#consider add more dense layer, decide on where the dropout should be 

5-element Array{String,1}:
 "3×3×1×1 AutoGrad.Param{KnetArray{Float32,4}}"  
 "4×4×1×1 AutoGrad.Param{KnetArray{Float32,4}}"  
 "5×5×1×1 AutoGrad.Param{KnetArray{Float32,4}}"  
 "1100×1802 AutoGrad.Param{KnetArray{Float32,2}}"
 "6×1100 AutoGrad.Param{KnetArray{Float32,2}}"   

In [22]:
cnn = trainresults("dcnn.jld2", dcnn);

Train from scratch? stdin> y
1.79e+00  0.15%┣                               ┫ 1/680 [00:12/02:13:57, 11.82s/i]

BoundsError: BoundsError: attempt to access 960-element UnitRange{Int32} at index [0]

## Experiment